In [1]:
import json
import imessages as im
from datetime import datetime

In [2]:
sender_id = "+15107503277"
# dt >= '2025-09-14' AND
# dt < '2025-09-15' AND
query_options = f"""
sender_id = '{sender_id}' AND
reply_to_guid IS NOT NULL AND
is_from_me = 1
"""

messages = im.get_messages(options=query_options)
print("Found", len(messages), "messages sent by you w/ a reply_to_guid")

cleaned_convos = im.get_conversations(messages, unique=False, clean=True, bad_messages_file='bad_messages.json')
print("Cleaned convos:", len(cleaned_convos))

Found 1631 messages sent by you w/ a reply_to_guid
Cleaned convos: 1628


In [5]:
bad_words = [
    # Racial slurs
    "nigga", "nigger", "chink",
    
    # Slurs targeting sexuality / gender
    "fag", "faggot", "tranny", "shemale", "homo",
    
    # Slurs targeting disability / mental health
    "retard", "sped", "autistic",
    
    # General profanity
    "bitch", "slut", "whore", "cunt", "fuck", "shit",

    # Adult / sexual / violent terms
    "sex", "porn", "erotic", "hentai", "nude", "naked",
    "boobs", "tits", "ass", "dick", "cock", "pussy",
    "orgasm", "cum", "jerk off", "jack off", "masturbate",
    "masturbation", "condom", "rape", "rapist", "raping",
    "kill",
]

def extract_bad_messages(convos, output_file=None):
    """
    convos: list of UNIQUE conversations
    """
    bad_messages = {}
    for convo in convos:
        for message in convo:
            for word in bad_words:
                if message['content'].lower().find(word) != -1:
                    bad_messages[message['guid']] = {
                        'bad_word': word,
                        'is_from_me': message['is_from_me'],
                        'content': message['content'],
                        'rephrased': message['content']
                    }
    
    if not output_file is None:
        with open(output_file, 'w') as f:
            json.dump(bad_messages, f, indent=4)
    
    return bad_messages

bad_messages = extract_bad_messages(cleaned_convos)
print('Found', len(bad_messages), 'bad messages')

Found 33 bad messages


In [4]:
im.prepare_convos_for_fine_tuning(cleaned_convos, output_file='fine_tuning_data.jsonl')

---

In [ ]:
def get_most_recent_message(messages, start_index, is_from_me=0):
    while not messages[start_index]["is_from_me"] == is_from_me:
        start_index -= 1
    return messages[start_index]

def calculate_time_diff(msg1, msg2):
    dt1 = datetime.strptime(msg1['dt'], '%Y-%m-%d %H:%M:%S')
    dt2 = datetime.strptime(msg2['dt'], '%Y-%m-%d %H:%M:%S')
    return (dt2 - dt1).total_seconds()

def prepare_for_fine_tuning(messages):
    system_prompt = "You are Sohan's texting bot. Your job is to reply exactly as Sohan would text his girlfriend, Ishani. If Ishani sends multiple texts in a row, you may also reply with multiple texts if needed. Never break character and always respond like Sohan texting his girlfriend."
    system_message = {"role": "system", "content": system_prompt}
    while messages[0]["is_from_me"]:
        del messages[0]

    fine_tuning_data = []
    i = 0

    while i < len(messages):
        if messages[i]["is_from_me"]:
            # prev_message = get_most_recent_message(messages, i, 0)
            # time_diff = calculate_time_diff(prev_message, messages[i])
            fine_tuning_data[-1]["messages"].append({"role": "assistant", "content": messages[i]["content"]})
        elif i > 0 and not messages[i-1]["is_from_me"]:
            fine_tuning_data[-1]["messages"].append({"role": "user", "content": messages[i]["content"]})
        else:
            fine_tuning_data.append({"messages": [
                system_message,
                {"role": "user", "content": messages[i]["content"]}
            ]})
        i += 1

    return fine_tuning_data

In [ ]:
fine_tuning_data = prepare_for_fine_tuning(messages)
fine_tuning_data[0]

{'messages': [{'role': 'system',
   'content': "You are Sohan's texting bot. Your job is to reply exactly as Sohan would text his girlfriend, Ishani. If Ishani sends multiple texts in a row, you may also reply with multiple texts if needed. Never break character and always respond like Sohan texting his girlfriend."},
  {'role': 'user', 'content': 'dude'},
  {'role': 'user', 'content': 'do u have car'},
  {'role': 'assistant', 'content': 'most likely yes'}]}

In [ ]:
with open('fine_tuning_data.jsonl', 'w') as f:
    for item in fine_tuning_data:
        f.write(json.dumps(item) + '\n')